<a href="https://colab.research.google.com/github/jared-ni/6.8610-project/blob/main/claude_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets
!pip install spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz
!pip install googletrans==4.0.0-rc1
!pip install deep-translator
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.6.1-py2.py3-n

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Create a DataFrame
df = pd.DataFrame({'Name': ['John', 'Mary'], 'Age': [25, 30]})

# Write to CSV
df.to_csv('/content/drive/My Drive/dummy_data.csv', index=False)

In [5]:
from google.colab import userdata
claude_token = userdata.get('claude_token')

In [6]:
import csv
from datasets import load_dataset
import pandas as pd
import spacy
from deep_translator import GoogleTranslator
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm

In [7]:
# Flags for which LLMs to use
USE_LLAMA = True
USE_MISTRAL = False
USE_FALCON = False

# Define max_length multiplier for LLM prompts
MAX_LENGTH_MULTIPLIER = 2

# Define k hyperparameter
K_HYPERPARAMETER = 3

In [23]:
# Load datasets into pandas DataFrames
def load_law_dataset():
    ds = load_dataset("casehold/casehold", "all")
    train_df = pd.DataFrame(ds['train'])
    test_df = pd.DataFrame(ds['test'])
    validation_df = pd.DataFrame(ds['validation'])
    law_dataset = pd.concat([train_df, test_df, validation_df], ignore_index=True)['citing_prompt']
    return law_dataset

def load_medical_dataset():
    ds = load_dataset("zhengyun21/PMC-Patients")
    train_df = pd.DataFrame(ds['train'])
    medical_dataset = train_df['patient']
    return medical_dataset

# Combine datasets
def load_all_datasets():
    law_dataset = load_law_dataset()
    medical_dataset = load_medical_dataset()
    return [law_dataset, medical_dataset]

In [24]:
# Load SpaCy model
def load_spacy_model(model_path='en_core_sci_sm'):
    return spacy.load(model_path)

# Extract entities from text
def extract_entities(nlp, text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents]

# Translate entities to a target language
def translate_entities(entities, target_lang):
    translations = [GoogleTranslator(source='auto', target=target_lang).translate(entity) for entity in entities]
    return translations

In [36]:
def calculate_JTC(translations, text, entities):
    jtc_score = 0
    n = len(entities)
    entity_counts = 0

    for entity, translated_entity in entities.items():
        # Count occurrences of the entity in the original text
        c = text.count(entity)
        entity_counts += c
        if c == 0 or not translated_entity:
            continue  # Skip entities that are not present or have no translation
        jargon_inconsistency = K_HYPERPARAMETER * c

        for translation in translations:
            # Count occurrences of the translated entity in the translation
            t = translation.count(translated_entity)
            # Calculate penalty for mismatched occurrences
            jargon_inconsistency -= t

        # Update the JTC score
        jtc_score += jargon_inconsistency

    # Normalize and invert the score
    normalized_score = jtc_score / max(K_HYPERPARAMETER * entity_counts, 1)
    return 1 - normalized_score

In [12]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have h

In [19]:
# import anthropic

# api_key = userdata.get('claude_token')
# client = anthropic.Anthropic(api_key=api_key)

# message = client.messages.create(
#     # model="claude-3-5-sonnet-20241022",
#     model="claude-3-haiku-20240307",
#     max_tokens=512,
#     messages=[
#         {"role": "user", "content": "Please translate this to Chinese: The patient was diagnosed with diabetes and is very sick"}
#     ]
# )
# print(message.content)

[TextBlock(text='该患者被诊断患有糖尿病,病情非常严重。', type='text')]


In [47]:
import anthropic

# Helper function to interact with Claude
def invoke_claude(prompt, model="claude-3-haiku-20240307", max_tokens=512):
    api_key = userdata.get('claude_token')
    client = anthropic.Anthropic(api_key=api_key)
    response = client.messages.create(
        model=model,
        max_tokens=max_tokens,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text

In [62]:
# Main function for the experiment
def run_pipeline_claude(target_lang, results_file_in_drive):
    datasets = load_all_datasets()
    lang_abbrs = {"Simplified Chinese": "zh-CN", "French": "fr"}

    # Path to the results file in Google Drive
    results_path = f'/content/drive/My Drive/{results_file_in_drive}'

    # Initialize the CSV file with headers
    with open(results_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Dataset", "Text Index", "Regular_JTC_Score", "LEAP_JTC_Score"])

    for dataset_index, dataset in enumerate(datasets):
        dataset_name = "law" if dataset_index == 0 else "medical"
        print(f"Processing dataset: {dataset_name}")

        for i, text in enumerate(dataset[:10]):
            text = " ".join(text.split()[:50])  # Truncate to the first 50 words
            print(f"Text {i+1}: {text}")
            named_entities = list(set(extract_entities(load_spacy_model(), text)))
            print("Named entities:", named_entities)
            named_entities_translations = translate_entities(named_entities, lang_abbrs[target_lang])
            print("Translations:", named_entities_translations)
            named_entity_mapping = {e: t for e, t in zip(named_entities, named_entities_translations)}

            # Regular text translation
            regular_translations = []
            for k in range(3):
                prompt = f"Please return only the answer and nothing else. Translate the following text to {target_lang}: {text}"
                response = invoke_claude(prompt)
                print("\nGenerated Response:", response)
                regular_translations.append(response)

            # LEAP text translation
            leap_translations = []
            for k in range(3):
                prompt = f"Please return only the answer and nothing else. Translate the following text to {target_lang} using these mappings {str(named_entities_translations)}: {text}"
                response = invoke_claude(prompt)
                print("\nGenerated Response (LEAP):", response)
                leap_translations.append(response)

            # Calculate JTC scores
            regular_jtc_score = calculate_JTC(regular_translations, text, named_entity_mapping)
            leap_jtc_score = calculate_JTC(leap_translations, text, named_entity_mapping)

            # Append result to the file in Google Drive
            with open(results_path, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                row = [dataset_name, i + 1, regular_jtc_score, leap_jtc_score]
                writer.writerow(row)
                print(f"Appended result to Google Drive file: {row}")

In [63]:
run_pipeline_claude("Simplified Chinese", "new_claude_results.csv")

Processing dataset: law
Text 1: Drapeau’s cohorts, the cohort would be a “victim” of making the bomb. Further, firebombs are inherently dangerous. There is no peaceful purpose for making a bomb. Felony offenses that involve explosives qualify as “violent crimes” for purposes of enhancing the sentences of career offenders. See 18 U.S.C. § 924(e)(2)(B)(ii) (defining
Named entities: ['U.S.C.', 'sentences', 'cohort', 'Drapeau’s cohorts', 'career', 'enhancing', 'offenders', '§', 'Felony', 'victim', 'violent crimes', 'bomb', 'firebombs']
Translations: ['美国加州大学', '句子', '同侪', '德拉波的同伙', '职业', '增强', '罪犯', None, '重罪', '受害者', '暴力犯罪', '炸弹', '燃烧弹']

Generated Response: 德拉波的同伙,该同伙将成为制造炸弹的"受害者"。此外,燃烧弹本质上是危险的。制造炸弹没有和平目的。涉及爆炸物的重罪构成"暴力犯罪",以加重惯犯的量刑。见18 U.S.C. § 924(e)(2)(B)(ii) (定义)。

Generated Response: 德拉波的同伙,这个同伙就会成为制造炸弹的"受害者"。此外,燃料弹也是固有的危险。制造炸弹没有和平目的。涉及爆炸物的重罪犯罪行为被视为"暴力犯罪",可以加重惯犯的刑期。

Generated Response: 德拉波的同伙,这个同伙会成为制造炸弹的"受害者"。此外,燃烧弹本质上是危险的。制造炸弹没有和平用途。涉及爆炸物的重罪构成"暴力犯罪",可以加重惯犯的刑期。参见18 U.S.C. § 924(e)(2